In [8]:
#import tensorflow as tf
#import keras
#from keras.layers import Input, Dense
#from keras.models import Model
#from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
#import os
#import h5py
#import pandas as pd

#import matplotlib.pyplot as plt

#import numpy as np
#from matplotlib.colors import LogNorm

In [23]:
#f = h5py.File('processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth.z', 'r')
#treeArray = f['t_allpar_new'][()]
#features = ['j1_ptrel','j1_etarot','j1_phirot','j1_erel','j1_deltaR','j1_costhetarel','j1_pdgid','j1_pt']
#labels = ['j_g','j_q','j_w','j_z','j_t']
#features_labels_df = pd.DataFrame(treeArray,columns = list(set(features + labels)))
#features_labels_df = features_labels_df.drop_duplicates()
#features_labels_df

#shuffle 
#features_val = features_labels_df[features].values
#labels_val = features_labels_df[labels].values

#X_train, X_test, y_train, y_test = train_test_split(features_val, labels_val, test_size=0.2, random_state=42)

In [20]:
import keras
from keras.models import load_model
from keras.models import model_from_json
from sklearn.metrics import roc_curve, auc
import numpy as np
import os
import h5py
import yaml
import pandas as pd
import matplotlib.pyplot as plt

from collections import namedtuple

Option = namedtuple("MyStruct", "inputModel inputFile tree config jsonModel ")
options = Option(
    inputModel = 'KERAS_conv1d_weights.h5',
    inputFile = 'processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth.z',
    tree = 't_allpar_new',
    config = 'train_config_conv1d.yml',
    jsonModel = 'KERAS_conv1d.json',
    #outputDir = 'output'
)

#print("Loading configuration from", options.config)
config = open(options.config, 'r')
#print(options.inputFile)
yamlConfig =  yaml.load(config, Loader = yaml.FullLoader)
yamlConfig

{'Inputs': ['j1_ptrel',
  'j1_etarot',
  'j1_phirot',
  'j1_erel',
  'j1_deltaR',
  'j1_costhetarel',
  'j1_pdgid'],
 'Labels': ['j_g', 'j_q', 'j_w', 'j_z', 'j_t'],
 'KerasModel': 'conv1d_model',
 'KerasModelRetrain': 'conv1d_model_constraint',
 'KerasLoss': 'categorical_crossentropy',
 'L1Reg': 0.0001,
 'NormalizeInputs': 1,
 'InputType': 'Conv1D',
 'MaxParticles': 100}

In [22]:
json_file = open(options.jsonModel, 'r')
print (options.jsonModel)
model = model_from_json(json_file.read())
#json_file.close()
#model.load_weights(options.inputModel)
#model.summary()

KERAS_conv1d.json


ValueError: Unknown constraint: ZeroSomeWeights

Load data


In [6]:
f = h5py.File(options.inputFile, 'r')
treeArray = f[options.tree][()]
#features = ['j1_ptrel','j1_etarot','j1_phirot','j1_erel','j1_deltaR','j1_costhetarel','j1_pdgid','j1_pt']
#labels = ['j_g','j_q','j_w','j_z','j_t']

features = yamlConfig['Inputs']
labels = yamlConfig['Labels']
#print(features)
#print(labels)

features_labels_df = pd.DataFrame(treeArray,columns = list(set(features + labels)))
features_labels_df = features_labels_df.drop_duplicates()
#print (features_labels_df)
features_labels_df

,j1_pdgid,j_w,j1_deltaR,j_z,j_g,j_t,j1_phirot,j_q,j1_ptrel,j1_etarot,j1_erel,j1_costhetarel
0,111.0,0,0.135378,1,0,0,-0.201170,0,0.001060,-0.068588,0.001084,-0.068480
1,111.0,0,0.321080,0,0,1,-0.348590,0,0.000626,-0.113826,0.000640,-0.113337
2,111.0,1,0.061471,0,0,0,0.001577,0,0.046280,0.003166,0.045609,0.003166
3,211.0,0,0.015611,0,1,0,-0.005436,0,0.132881,-0.015675,0.133103,-0.015673
4,211.0,0,0.328146,0,0,1,-0.006246,0,0.001762,-0.350233,0.001496,-0.336582
...,...,...,...,...,...,...,...,...,...,...,...,...
5119904,111.0,0,0.071287,0,1,0,-0.104943,0,0.003359,0.061939,0.003480,0.061860
5119905,111.0,1,0.243789,0,0,0,-0.271796,0,0.003650,0.003421,0.004210,0.003421
5119906,2212.0,0,0.205047,1,0,0,-0.242324,0,0.022742,-0.002743,0.022763,-0.002743
5119907,111.0,0,0.215792,0,1,0,-0.163521,0,0.002943,-0.199286,0.003270,-0.196689


In [9]:
features_val = features_labels_df[features].values
labels_val = features_labels_df[labels].values
X_train, X_test, y_train, y_test = train_test_split(features_val, labels_val, test_size=0.2, random_state=42)

In [6]:
adam = Adam(lr = 0.0001)
model.compile(optimizer=adam, loss=[yamlConfig['KerasLoss']], metrics=['accuracy'])

NameError: name 'model' is not defined

In [ ]:
history = model.fit(X_train, y_train, batch_size = 1024, epochs = 100, 
                    validation_split = 0.25, shuffle = True, callbacks = None,
                    use_multiprocessing=True, workers=4)